# Twitter API

#### STREAMING API ~ TWEEPY

In [1]:
import tweepy
from sqlalchemy.exc import ProgrammingError
import json
import cnfg
from requests_oauthlib import OAuth1
import pandas as pd
import pymongo
import json
from pymongo import MongoClient

In [2]:
config = cnfg.load("twitter_config/.twitter_config")
auth = tweepy.OAuthHandler(config["consumer_key"],
                           config["consumer_secret"])

auth.set_access_token(config["access_token"],
                      config["access_token_secret"])

api=tweepy.API(auth)

### Live Streaming & Mongodb

In [3]:
client = MongoClient(port=27017)
db = client.example_live
livetweets = db.anthem

livetweets.drop() # only if we want to start from scratch

class StreamListener(tweepy.StreamListener):

#     def __init__(self, start_time, time_limit=10):
 
#         self.time = start_time
#         self.limit = time_limit
#         self.tweet_data = []
    
    def on_status(self, status):
        
        # skip retweets
        if status.retweeted: 
            return
    
        # store tweet and creation date
        data = {}
        data['tweet'] = status.text
        data['datetime'] = status.created_at
        #data['location'] = status.place_id

        # insert into db
        try:
            livetweets.insert_one(data)
        except:
            pass

        
user = api.get_user('twitter')
stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
stream.filter(track=['nfl','NFL','national anthem','National Anthem'])

ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.

In [4]:
livetweets.count()

/Users/andreacsorcinelli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


2545

In [5]:
livetweets.find_one()

{'_id': ObjectId('5b808e7482816f41cb1aef38'),
 'tweet': 'RT @BISHPHAT: Enter for my $5 Slow Draft Bestball Football League now on the Draft app. \n\nLeague Name: Fantasy Assembly \n\nWe will Draft whe…',
 'datetime': datetime.datetime(2018, 8, 24, 23, 2, 22)}

In [6]:
df = pd.DataFrame(list(livetweets.find()))

In [7]:
df.head()

,_id,datetime,tweet
0,5b808e7482816f41cb1aef38,2018-08-24 23:02:22,RT @BISHPHAT: Enter for my $5 Slow Draft Bestb...
1,5b808e7582816f41cb1aef39,2018-08-24 23:02:22,Dude I remember this like this was yesterday m...
2,5b808e7582816f41cb1aef3a,2018-08-24 23:02:22,RT @NFL: TE @Julius_Thomas retiring to pursue ...
3,5b808e7682816f41cb1aef3b,2018-08-24 23:02:23,RT @NFL: .@OBJ_3 for the kids! 😁\n\n#NYGvsNYJ ...
4,5b808e7682816f41cb1aef3c,2018-08-24 23:02:24,NFL better come through because I need to meet...


In [8]:
df.shape

(2545, 3)